# Melhor modelo LSTM padrao ate o momento R2 de .71

In [10]:
from sklearn.model_selection import TimeSeriesSplit
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from sympy import false
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.impute import SimpleImputer, KNNImputer
from copy import deepcopy as dc
import logging
from datetime import datetime
import optuna
import torch.nn.functional as F
import os

In [11]:
# Configuração inicial
data_hoje = datetime.now().strftime('%d-%m')
inicio_execucao = pd.Timestamp.now()

# Criando diretórios para logs e plots
os.makedirs(f'../logs/{data_hoje}', exist_ok=True)
os.makedirs(f'../plots/{data_hoje}', exist_ok=True)

# Configuração do logging
logging.basicConfig(filename=f'../logs/{data_hoje}/lstm_optuna.log', level=logging.INFO, format='- %(message)s')
logging.info('-' * 50)
logging.info(f'{inicio_execucao} - Iniciando o processo de otimização e treinamento do modelo LSTM')

# Carregando e preparando os dados
df_original = pd.read_csv('../dados_tratados/combinado/Piratininga/Piratininga_tratado_combinado.csv',
                          usecols=['PM2.5', 'Data e Hora', 'PM10', 'Monóxido de Carbono', 'Dióxido de Enxofre',
                                   'Dióxido de Nitrogênio', 'Temperatura', 'Velocidade do Vento', 'Umidade Relativa',
                                   'Direção do Vento'], low_memory=False)

df_original['Data e Hora'] = pd.to_datetime(df_original['Data e Hora'])
df_original.set_index('Data e Hora', inplace=True)
df_original.sort_index(inplace=True)

colunas_selecionadas = ['PM2.5', 'PM10', 'Monóxido de Carbono']
logging.info(f"Colunas selecionadas: {colunas_selecionadas}")
df = df_original[colunas_selecionadas]
df = df.loc['2019-01-01':'2022-01-01']

df = df.apply(pd.to_numeric, errors='coerce')

In [12]:
# fazendo o logging de qual algoritmo de imputação foi utilizado
def log_imputation(method_name, impute_function, df):
    df_imputed = impute_function(df)
    logging.info(f"Imputação realizada usando: {method_name}")
    return df_imputed


def linear_interpolation_imputer(df):
    df_imputed = df.interpolate(method='linear')
    return df_imputed


df_imputed = log_imputation('Linear Interpolation', linear_interpolation_imputer, df)

logging.info(f"Dados ausentes antes da imputação: {df.isna().sum()}")
logging.info(f"Dados ausentes após a imputação: {df_imputed.isna().sum()}")
logging.info(f"Dados totais: {len(df_imputed)}")

In [13]:
from sklearn.preprocessing import StandardScaler


# Preparando os dados para LSTM
def prepare_dataframe_for_lstm(df, n_steps):
    df = dc(df)
    for col in colunas_selecionadas:
        for i in range(1, n_steps + 1):
            df[f'{col}(t-{i})'] = df[col].shift(i)
    df.dropna(inplace=True)
    return df


lookback = 168  # 1 semana
shifted_df = prepare_dataframe_for_lstm(df_imputed, lookback)

# Dividindo em conjuntos de treino, validação e teste
train_size = int(len(shifted_df) * 0.7)
val_size = int(len(shifted_df) * 0.15)

train_df = shifted_df.iloc[:train_size]
val_df = shifted_df.iloc[train_size:train_size + val_size]
test_df = shifted_df.iloc[train_size + val_size:]

# Normalizando os dados de forma correta
scaler = StandardScaler()
train_scaled = pd.DataFrame(scaler.fit_transform(train_df), columns=shifted_df.columns, index=train_df.index)
val_scaled = pd.DataFrame(scaler.transform(val_df), columns=shifted_df.columns, index=val_df.index)
test_scaled = pd.DataFrame(scaler.transform(test_df), columns=shifted_df.columns, index=test_df.index)

X_train, y_train = train_scaled.iloc[:, len(colunas_selecionadas):].values, train_scaled.iloc[:, 0].values
X_val, y_val = val_scaled.iloc[:, len(colunas_selecionadas):].values, val_scaled.iloc[:, 0].values
X_test, y_test = test_scaled.iloc[:, len(colunas_selecionadas):].values, test_scaled.iloc[:, 0].values

# Reshape para LSTM
X_train = X_train.reshape((-1, lookback, len(colunas_selecionadas)))
X_val = X_val.reshape((-1, lookback, len(colunas_selecionadas)))
X_test = X_test.reshape((-1, lookback, len(colunas_selecionadas)))
y_train = y_train.reshape((-1, 1))
y_val = y_val.reshape((-1, 1))
y_test = y_test.reshape((-1, 1))

# Convertendo para tensores PyTorch
X_train = torch.tensor(X_train).float()
y_train = torch.tensor(y_train).float()
X_val = torch.tensor(X_val).float()
y_val = torch.tensor(y_val).float()
X_test = torch.tensor(X_test).float()
y_test = torch.tensor(y_test).float()


# Dataset e DataLoader
class TimeSeriesDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, i):
        return self.X[i], self.y[i]


# Modelo LSTM
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

C:\Users\portes\AppData\Local\Temp\ipykernel_12508\3031914985.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}(t-{i})'] = df[col].shift(i)
C:\Users\portes\AppData\Local\Temp\ipykernel_12508\3031914985.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}(t-{i})'] = df[col].shift(i)
C:\Users\portes\AppData\Local\Temp\ipykernel_12508\3031914985.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider join

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Attention(nn.Module):
    def __init__(self, hidden_size):
        super(Attention, self).__init__()
        self.attention_weights = nn.Parameter(torch.randn(hidden_size))

    def forward(self, lstm_output):
        attention_scores = torch.tanh(torch.matmul(lstm_output, self.attention_weights))
        attention_scores = F.softmax(attention_scores, dim=1)
        weighted_output = torch.mul(lstm_output, attention_scores.unsqueeze(-1))
        return weighted_output.sum(dim=1)


class LSTM_MLP(nn.Module):
    def __init__(self, input_size, hidden_sizes, mlp_sizes, activation, dropout):
        super().__init__()
        self.hidden_sizes = hidden_sizes
        self.num_lstm_layers = len(hidden_sizes)
        self.activation = activation

        # LSTM Encoder
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_sizes[-1],
            num_layers=self.num_lstm_layers,
            batch_first=True,
            dropout=dropout if self.num_lstm_layers > 1 else 0
        )

        # Attention mechanism
        self.attention = Attention(hidden_sizes[-1])

        # MLP Decoder
        mlp_layers = []
        in_features = hidden_sizes[-1]
        for out_features in mlp_sizes:
            mlp_layers.append(nn.Linear(in_features, out_features))
            mlp_layers.append(self.get_activation())
            mlp_layers.append(nn.Dropout(dropout))
            in_features = out_features
        mlp_layers.append(nn.Linear(in_features, 1))  # Output layer
        self.mlp = nn.Sequential(*mlp_layers)

    def forward(self, x):
        # LSTM Encoder
        lstm_out, _ = self.lstm(x)

        # Attention
        attended_output = self.attention(lstm_out)

        # MLP Decoder
        out = self.mlp(attended_output)
        return out

    def get_activation(self):
        if self.activation == 'relu':
            return nn.ReLU()
        elif self.activation == 'leaky_relu':
            return nn.LeakyReLU()
        elif self.activation == 'elu':
            return nn.ELU()
        elif self.activation == 'sigmoid':
            return nn.Sigmoid()
        elif self.activation == 'tanh':
            return nn.Tanh()
        elif self.activation == 'swish':
            return nn.SiLU()  # PyTorch's implementation of Swish
        elif self.activation == 'mish':
            return nn.Mish()
        elif self.activation == 'gelu':
            return nn.GELU()
        elif self.activation == 'softplus':
            return nn.Softplus()
        else:
            return nn.Identity()


In [15]:
from torch.nn.utils import clip_grad_norm_
from torch.optim.lr_scheduler import ReduceLROnPlateau


# Função de treinamento
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, patience, device):
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)

    best_val_loss = float('inf')
    epochs_without_improvement = 0
    best_model_state = None

    for epoch in range(num_epochs):
        # Treinamento
        model.train()
        train_loss = 0.0
        for batch in train_loader:
            x_batch, y_batch = batch[0].to(device), batch[1].to(device)
            optimizer.zero_grad()
            output = model(x_batch)
            loss = criterion(output, y_batch)
            
            l1_lambda = 0.01
            l1_norm = sum(p.abs().sum() for p in model.parameters())
            loss += l1_lambda * l1_norm
            loss.backward()

            # Gradient clipping
            clip_grad_norm_(model.parameters(), max_norm=1.0)

            optimizer.step()
            train_loss += loss.item()

        train_loss /= len(train_loader)

        # Validação
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for batch in val_loader:
                x_batch, y_batch = batch[0].to(device), batch[1].to(device)
                output = model(x_batch)
                val_loss += criterion(output, y_batch).item()
        val_loss /= len(val_loader)

        # Ajuste da taxa de aprendizagem
        scheduler.step(val_loss)

        # logging.info(f"Época {epoch + 1}/{num_epochs}, Perda de treino: {train_loss:.4f}, Perda de validação: {val_loss:.4f}")

        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            epochs_without_improvement = 0
            best_model_state = model.state_dict()
        else:
            epochs_without_improvement += 1

        if epochs_without_improvement >= patience:
            logging.info(f"Early stopping ativado na época {epoch + 1}")
            break

    # Carregar o melhor modelo
    model.load_state_dict(best_model_state)
    return model

In [16]:
from sklearn.metrics import mean_absolute_percentage_error


# Cálculo das métricas
def calculate_metrics(y_true, y_pred):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    return rmse, mse, mae, r2, mape

In [17]:
def objective(trial):
    # Hiperparâmetros para otimização
    num_lstm_layers = trial.suggest_int('num_lstm_layers', 1, 6)
    hidden_sizes = [trial.suggest_int(f'hidden_size_{i}', 32, 256) for i in range(num_lstm_layers)]

    num_mlp_layers = trial.suggest_int('num_mlp_layers', 1, 4)
    mlp_sizes = [trial.suggest_int(f'mlp_size_{i}', 32, 256) for i in range(num_mlp_layers)]

    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
    learning_rate = trial.suggest_float('learning_rate', 1e-6, 1e-2, log=True)
    activation = trial.suggest_categorical('activation',
                                           ['relu', 'leaky_relu', 'elu', 'sigmoid', 'swish', 'mish', 'gelu', 'tanh',
                                            'softplus'])
    dropout = trial.suggest_float('dropout', 0.0, 0.5)
    weight_decay = trial.suggest_float('weight_decay', 1e-6, 1e-3, log=True)

    # Criação dos DataLoaders
    train_dataset = TimeSeriesDataset(X_train, y_train)
    val_dataset = TimeSeriesDataset(X_val, y_val)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)

    # Create and train the model
    model = LSTM_MLP(len(colunas_selecionadas), hidden_sizes, mlp_sizes, activation, dropout).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    criterion = nn.HuberLoss()

    num_epochs = 1000
    early_stopping_patience = 20

    model = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, early_stopping_patience,
                        device)

    # Avaliação no conjunto de validação
    model.eval()
    val_loss = 0.0
    val_predictions = []
    val_true = []
    with torch.no_grad():
        for batch in val_loader:
            x_batch, y_batch = batch[0].to(device), batch[1].to(device)
            output = model(x_batch)
            val_loss += criterion(output, y_batch).item()
            val_predictions.extend(output.cpu().numpy())
            val_true.extend(y_batch.cpu().numpy())
    val_loss /= len(val_loader)

    # Calcular métricas
    val_predictions = np.array(val_predictions).flatten()
    val_true = np.array(val_true).flatten()
    rmse, mse, mae, r2, mape = calculate_metrics(val_true, val_predictions)

    # Imprimir métricas
    print(f"Trial {trial.number}:")
    print(f"  MAPE: {mape:.4f}")
    print(f"  RMSE: {rmse:.4f}")
    print(f"  R²: {r2:.4f}")
    print(f"  Validation Loss: {val_loss:.4f}")

    # Salvar métricas como atributos do trial
    trial.set_user_attr("MAPE", mape)
    trial.set_user_attr("RMSE", rmse)
    trial.set_user_attr("R2", r2)

    return rmse


In [18]:
torch.backends.cudnn.benchmark = True

study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler())
study.optimize(objective, n_trials=10)

# Após a otimização, imprimir os melhores resultados
best_trial = study.best_trial
print("Best trial:")
print(f"  Value (Validation Loss): {best_trial.value:.4f}")
print(f"  MAPE: {best_trial.user_attrs['MAPE']:.4f}")
print(f"  RMSE: {best_trial.user_attrs['RMSE']:.4f}")
print(f"  R²: {best_trial.user_attrs['R2']:.4f}")
print("  Params: ")
for key, value in best_trial.params.items():
    print(f"    {key}: {value}")

[I 2024-09-16 17:33:08,524] A new study created in memory with name: no-name-fda45177-08dc-49d5-bf10-2a6f626fb6ca
C:\dev\scripts\pm25-plots\venv\lib\site-packages\torch\optim\lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
[I 2024-09-16 17:42:02,413] Trial 0 finished with value: 0.9062961935997009 and parameters: {'num_lstm_layers': 2, 'hidden_size_0': 60, 'hidden_size_1': 101, 'num_mlp_layers': 2, 'mlp_size_0': 59, 'mlp_size_1': 63, 'batch_size': 64, 'learning_rate': 0.009404354306507543, 'activation': 'relu', 'dropout': 0.26596610029152756, 'weight_decay': 1.2147144914405277e-05}. Best is trial 0 with value: 0.9062961935997009.


Trial 0:
  MAPE: 108.5867
  RMSE: 0.9063
  R²: -0.0681
  Validation Loss: 0.2845


C:\dev\scripts\pm25-plots\venv\lib\site-packages\torch\optim\lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
[I 2024-09-16 17:47:03,444] Trial 1 finished with value: 0.885482668876648 and parameters: {'num_lstm_layers': 2, 'hidden_size_0': 152, 'hidden_size_1': 142, 'num_mlp_layers': 3, 'mlp_size_0': 146, 'mlp_size_1': 170, 'mlp_size_2': 175, 'batch_size': 128, 'learning_rate': 6.0428450961986895e-05, 'activation': 'swish', 'dropout': 0.42038163435985826, 'weight_decay': 7.045713389999384e-05}. Best is trial 1 with value: 0.885482668876648.


Trial 1:
  MAPE: 94.8971
  RMSE: 0.8855
  R²: -0.0196
  Validation Loss: 0.2803


C:\dev\scripts\pm25-plots\venv\lib\site-packages\torch\optim\lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
[I 2024-09-16 18:02:28,504] Trial 2 finished with value: 0.8958740234375 and parameters: {'num_lstm_layers': 6, 'hidden_size_0': 251, 'hidden_size_1': 107, 'hidden_size_2': 97, 'hidden_size_3': 75, 'hidden_size_4': 49, 'hidden_size_5': 238, 'num_mlp_layers': 2, 'mlp_size_0': 218, 'mlp_size_1': 73, 'batch_size': 32, 'learning_rate': 1.1424980301473202e-05, 'activation': 'relu', 'dropout': 0.4460378269226014, 'weight_decay': 2.1306454872763194e-06}. Best is trial 1 with value: 0.885482668876648.


Trial 2:
  MAPE: 100.9102
  RMSE: 0.8959
  R²: -0.0437
  Validation Loss: 0.2816


C:\dev\scripts\pm25-plots\venv\lib\site-packages\torch\optim\lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
[I 2024-09-16 18:03:11,866] Trial 3 finished with value: 0.886960506439209 and parameters: {'num_lstm_layers': 3, 'hidden_size_0': 194, 'hidden_size_1': 217, 'hidden_size_2': 93, 'num_mlp_layers': 3, 'mlp_size_0': 104, 'mlp_size_1': 132, 'mlp_size_2': 79, 'batch_size': 256, 'learning_rate': 0.00019690279927483872, 'activation': 'mish', 'dropout': 0.0651091938840076, 'weight_decay': 7.541373410497123e-05}. Best is trial 1 with value: 0.885482668876648.


Trial 3:
  MAPE: 93.2174
  RMSE: 0.8870
  R²: -0.0230
  Validation Loss: 0.2829


C:\dev\scripts\pm25-plots\venv\lib\site-packages\torch\optim\lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
[W 2024-09-16 18:05:51,723] Trial 4 failed with parameters: {'num_lstm_layers': 3, 'hidden_size_0': 154, 'hidden_size_1': 245, 'hidden_size_2': 153, 'num_mlp_layers': 2, 'mlp_size_0': 177, 'mlp_size_1': 112, 'batch_size': 128, 'learning_rate': 2.2326598486077998e-06, 'activation': 'leaky_relu', 'dropout': 0.16480310047969737, 'weight_decay': 3.134288755474082e-05} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "C:\dev\scripts\pm25-plots\venv\lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\portes\AppData\Local\Temp\ipykernel_12508\2794210514.py", line 31, in objective
    model = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, early_stoppi

KeyboardInterrupt: 

In [ ]:
best_params = study.best_params
logging.info(f"Melhores hiperparâmetros: {best_params}")

# Treinamento final com os melhores hiperparâmetros
best_hidden_sizes = [best_params[f'hidden_size_{i}'] for i in range(best_params['num_layers'])]
best_batch_size = best_params['batch_size']

train_dataset = TimeSeriesDataset(X_train, y_train)
val_dataset = TimeSeriesDataset(X_val, y_val)
test_dataset = TimeSeriesDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=best_batch_size, shuffle=False, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=best_batch_size, shuffle=False, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=best_batch_size, shuffle=False, pin_memory=True)

final_model = LSTM_MLP(len(colunas_selecionadas), best_hidden_sizes, best_params['activation'], best_params['dropout']).to(
    device)
optimizer = torch.optim.AdamW(final_model.parameters(), lr=best_params['learning_rate'])
criterion = nn.HuberLoss()

num_epochs = 1000
patience = 20

final_model = train_model(final_model, train_loader, val_loader, criterion, optimizer, num_epochs, patience, device)

# Salvar o modelo final
torch.save(final_model.state_dict(), f'../models/best_model_optuna_{data_hoje}.pth')


# Avaliação final
def evaluate(model, dataloader):
    predictions = []
    actual = []
    model.eval()
    with torch.no_grad():
        for batch in dataloader:
            x_batch, y_batch = batch[0].to(device), batch[1].to(device)
            output = model(x_batch)
            predictions.extend(output.cpu().numpy().flatten())
            actual.extend(y_batch.cpu().numpy().flatten())
    return np.array(predictions), np.array(actual)


train_predictions, train_actual = evaluate(final_model, train_loader)
val_predictions, val_actual = evaluate(final_model, val_loader)
test_predictions, test_actual = evaluate(final_model, test_loader)

In [ ]:
# Desnormalização
def inverse_transform_data(normalized_data):
    # Criar um array de zeros com o mesmo número de colunas que os dados originais
    dummy_array = np.zeros((len(normalized_data), len(shifted_df.columns)))

    # Colocar os dados normalizados na primeira coluna (assumindo que é PM2.5)
    dummy_array[:, 0] = normalized_data

    # Aplicar a transformação inversa
    denormalized_data = scaler.inverse_transform(dummy_array)

    # Retornar apenas a primeira coluna, que contém os dados desnormalizados de interesse
    return denormalized_data[:, 0]


# Desnormalização das previsões e valores reais
train_predictions = inverse_transform_data(train_predictions)
val_predictions = inverse_transform_data(val_predictions)
test_predictions = inverse_transform_data(test_predictions)

# Para os valores reais, precisamos garantir que estamos usando os dados originais não normalizados
train_actual = df_imputed['PM2.5'].values[:len(train_predictions)]
val_actual = df_imputed['PM2.5'].values[len(train_predictions):len(train_predictions) + len(val_predictions)]
test_actual = df_imputed['PM2.5'].values[-len(test_predictions):]

# Calcular métricas
train_rmse, train_mse, train_mae, train_r2, train_mape = calculate_metrics(train_actual, train_predictions)
val_rmse, val_mse, val_mae, val_r2, val_mape = calculate_metrics(val_actual, val_predictions)
test_rmse, test_mse, test_mae, test_r2, test_mape = calculate_metrics(test_actual, test_predictions)

# Logging dos resultados
logging.info(
    f"Métricas de Treino: RMSE={train_rmse:.4f}, MSE={train_mse:.4f}, MAE={train_mae:.4f}, R2={train_r2:.4f}, MAPE={train_mape:.4f}")
logging.info(
    f"Métricas de Validação: RMSE={val_rmse:.4f}, MSE={val_mse:.4f}, MAE={val_mae:.4f}, R2={val_r2:.4f}, MAPE={val_mape:.4f}")
logging.info(
    f"Métricas de Teste: RMSE={test_rmse:.4f}, MSE={test_mse:.4f}, MAE={test_mae:.4f}, R2={test_r2:.4f}, MAPE={test_mape:.4f}")

print(
    f"Métricas de Treino: RMSE={test_rmse:.4f}, MSE={test_mse:.4f}, MAE={test_mae:.4f}, R2={test_r2:.4f}, MAPE={test_mape:.4f}")

In [ ]:

# Log das métricas finais
logging.info("\nMétricas finais:")
logging.info("Treinamento - RMSE: {:.4f}, MSE: {:.4f}, MAE: {:.4f}, R²: {:.4f}".format(train_rmse, train_mse, train_mae,
                                                                                       train_r2))
logging.info(
    "Validação - RMSE: {:.4f}, MSE: {:.4f}, MAE: {:.4f}, R²: {:.4f}".format(val_rmse, val_mse, val_mae, val_r2))
logging.info(
    "Teste - RMSE: {:.4f}, MSE: {:.4f}, MAE: {:.4f}, R²: {:.4f}".format(test_rmse, test_mse, test_mae, test_r2))

# Plotagem dos resultados
plt.figure(figsize=(12, 6))
plt.plot(train_actual, label='Actual PM2.5')
plt.plot(train_predictions, label='Predicted PM2.5')
plt.title('Treinamento: PM2.5 Real vs Previsto')
plt.xlabel('Hora')
plt.ylabel('PM2.5')
plt.legend()
plt.savefig(f'../plots/{data_hoje}/lstm_optuna_train_{data_hoje}.png')


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

train_dates = shifted_df.index[:len(train_actual)]
val_dates = shifted_df.index[len(train_actual):len(train_actual) + len(val_actual)]
test_dates = shifted_df.index[-len(test_actual):]


def plot_results(actual, predicted, dates, title, filename):
    plt.figure(figsize=(20, 12))
    plt.plot(dates, actual, label='Real', color='blue')
    plt.plot(dates, predicted, label='Previsto', color='red')
    plt.title(title)
    plt.xlabel('Data')
    plt.ylabel('PM2.5')
    plt.legend()

    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
    plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=1))  # Mostrar a cada 3 meses

    plt.gcf().autofmt_xdate()  # Rotacionar e alinhar os rótulos de data
    plt.tight_layout()
    plt.savefig(f'../plots/{data_hoje}/{filename}_{data_hoje}.png')
    plt.close()


plot_results(train_actual, train_predictions, train_dates, 'Treinamento: PM2.5 Real vs Previsto', 'lstm_optuna_train')
plot_results(val_actual, val_predictions, val_dates, 'Validação: PM2.5 Real vs Previsto', 'lstm_optuna_val')
plot_results(test_actual, test_predictions, test_dates, 'Teste: PM2.5 Real vs Previsto', 'lstm_optuna_test')

fim_execucao = pd.Timestamp.now()
tempo_execucao = fim_execucao - inicio_execucao
logging.info(f"\nExecução finalizada em {fim_execucao}")
logging.info(f"Tempo total de execução: {tempo_execucao}")

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd


def plot_results_by_month(actual, predicted, dates, title_prefix, filename_prefix):
    df = pd.DataFrame({'date': dates, 'actual': actual, 'predicted': predicted})
    df.set_index('date', inplace=True)

    grouped = df.groupby(pd.Grouper(freq='M'))

    for name, group in grouped:
        if len(group) > 0:
            plt.figure(figsize=(12, 6))
            plt.plot(group.index, group['actual'], label='Real', color='blue')
            plt.plot(group.index, group['predicted'], label='Previsto', color='red')

            month_year = name.strftime('%B %Y')
            plt.title(f'{title_prefix} - {month_year}')
            plt.xlabel('Data')
            plt.ylabel('PM2.5')
            plt.legend()

            plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%d-%m'))
            plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=5))

            plt.gcf().autofmt_xdate()  # Rotacionar e alinhar os rótulos de data
            plt.tight_layout()

            month_filename = f'{filename_prefix}_{name.strftime("%Y_%m")}_{data_hoje}.png'
            plt.savefig(f'../plots/{data_hoje}/{month_filename}')
            plt.close()


plot_results_by_month(train_actual, train_predictions, train_dates, 'Treinamento: PM2.5 Real vs Previsto',
                      'lstm_optuna_train')
plot_results_by_month(val_actual, val_predictions, val_dates, 'Validação: PM2.5 Real vs Previsto', 'lstm_optuna_val')
plot_results_by_month(test_actual, test_predictions, test_dates, 'Teste: PM2.5 Real vs Previsto', 'lstm_optuna_test')